# Importing modules

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re,string
import tensorflow as tf
import enchant
import trafilatura

from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from nltk.corpus import stopwords

# Importing Data

In [2]:
true = pd.read_csv('../raw_data/True.csv')
fake = pd.read_csv('../raw_data/Fake.csv')

## Deleting non necessary columns

In [3]:
true.drop(columns = ['subject','date'], inplace = True)
fake.drop(columns = ['subject','date'], inplace = True)

## Brief data cleaning to fake dataset

In [4]:
stop_words = ['/Getty Images']
pat = '|'.join(r"\b{}\b".format(x) for x in stop_words)
fake['text'] = fake['text'].str.replace(pat, '')

## Creating Target

In [5]:
true['score'] = 1
fake['score'] = 0

## Concatenate everything to one dataset

In [6]:
data = pd.concat([true,fake],ignore_index=True)

# Cleaning some data

## Removing links from text

In [7]:
import re
def rem_urls(text):
    return re.sub('https?:\S+','',text)

In [8]:
#data['title_clean']=data['title'].apply(rem_urls)
#data['text_clean']=data['text'].apply(rem_urls)

## Removing punctuation 

In [9]:
import string

punc_no_sq = '!“#$%&\()*+,./:;<=>?@[\\]^_`{|}~“”—’-'

def remove_punctuation(text):
    for punctuation in punc_no_sq:
        text = text.replace(punctuation, '')
    return text

In [10]:
#data['title_clean']=data['title_clean'].apply(remove_punctuation)
#data['text_clean']=data['text_clean'].apply(remove_punctuation)

## Removing Numbers

In [11]:
def remove_numbers(text):
    text = ''.join(word for word in text if not word.isdigit())
    return text

In [12]:
#data['title_clean']=data['title_clean'].apply(remove_numbers)

#data['text_clean']=data['text_clean'].apply(remove_numbers)

## Making everything lower case

In [13]:
def lower_case(text):
    text = text.lower()
    return text

In [14]:
#data['title_clean']=data['title_clean'].apply(lower_case)
#data['text_clean']=data['text_clean'].apply(lower_case)

## Removing Stop Words

In [15]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

#data['title_clean'] = data['title_clean'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop_words)]))
#data['text_clean'] = data['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop_words)]))

## tokenizing text in order to count how many words we have to calculate ratio

In [16]:
def tokenize_text(text):
    return text.split()

#data['title_tokens']=data['title_clean'].apply(tokenize_text)
#data['text_tokens']=data['text_clean'].apply(tokenize_text)

## Function to count typos in text

In [17]:
english = enchant.DictWithPWL("en_US", "vocab.txt")
wrong_words={}
correct_words=set()
def get_typos_t(tokens):
     wrong_count=0
     for token in tokens:
            if token in wrong_words:
                wrong_words[token]+=1
                wrong_count+=1
            else:
                if not token in correct_words:
                    if token[0].islower() and not '-' in token and not english.check(token) and not english.check(token.capitalize()):
                        wrong_words[token]=1
                        wrong_count+=1
                    else:
                        correct_words.add(token)
     return wrong_count    

# Create typo ratio

In [18]:
#data['typos_title_count']=data['title_tokens'].apply(get_typos_t)
#data['typos_text_count']=data['text_tokens'].apply(get_typos_t)

In [19]:
#data['title_typo_ratio']= data['typos_title_count']/len(data['title_tokens'])
#data['text_typo_ratio']= data['typos_text_count']/len(data['text_tokens'])

# Defining one cleaning function

In [20]:
#data = data[['title', 'text', 'title_clean', 'text_clean', 'title_tokens',
      # 'text_tokens', 'typos_title_count', 'typos_text_count',
      # 'title_typo_ratio', 'text_typo_ratio','score']]

In [1]:
import re
def rem_urls(text):
    return re.sub('https?:\S+','',text)

import string
punc_no_sq = '!“#$%&\()*+,./:;<=>?@[\\]^_`{|}~“”—’-'
def remove_punctuation(text):
    for punctuation in punc_no_sq:
        text = text.replace(punctuation, '')
    return text

def remove_numbers(text):
    text = ''.join(word for word in text if not word.isdigit())
    return text

def lower_case(text):
    text = text.lower()
    return text

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


def tokenize_text(text):
    return text.split()

from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def stopwords_ratio(tokens):
    count_stop_words = 0
    amount_tokens = 0   
    for token in tokens:
        amount_tokens += 1
        if token in stop_words:
            count_stop_words += 1
    if amount_tokens == 0:
        return 0
    return count_stop_words / amount_tokens

import enchant
english = enchant.DictWithPWL("en_US", "vocab.txt")
wrong_words={}
correct_words=set()
def get_typos_t(tokens):
     wrong_count=0
     for token in tokens:
            if token in wrong_words:
                wrong_words[token]+=1
                wrong_count+=1
            else:
                if not token in correct_words:
                    if token[0].islower() and not '-' in token and not english.check(token) and not english.check(token.capitalize()):
                        wrong_words[token]=1
                        wrong_count+=1
                    else:
                        correct_words.add(token)
     return wrong_count 


def clean_data(data):
    
    df = data.copy()
    
    #drop nan values in df
    
    df.dropna(axis = 0, inplace = True)
    
    #add title_clean and text_clean to df
    stop_words = ['reuters']    
    pat = '|'.join(r"\b{}\b".format(x) for x in stop_words)
    
    df['title_clean'] = df['title'].apply(rem_urls)
    df['text_clean_without_reuters'] = df['text'].apply(rem_urls)
    df['text_clean_without_reuters'] = df['text_clean_without_reuters'].str.replace(pat, '')
    df['title_clean'] = df['title_clean'].apply(remove_punctuation)
    df['text_clean_without_reuters'] = df['text_clean_without_reuters'].apply(remove_punctuation)
    df['title_clean'] = df['title_clean'].apply(remove_numbers)
    df['text_clean_without_reuters'] = df['text_clean_without_reuters'].apply(remove_numbers)
    df['title_clean'] = df['title_clean'].apply(lower_case)
    df['text_clean_without_reuters'] = df['text_clean_without_reuters'].apply(lower_case)
    df['title_clean'] = df['title_clean'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop_words)]))
    df['text_clean_without_reuters'] = df['text_clean_without_reuters'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop_words)]))
    
    #adding features to the df
    
    df['title_tokens'] = df['title_clean'].apply(tokenize_text)
    df['text_tokens'] = df['text_clean_without_reuters'].apply(tokenize_text)
    
    df['text_stop_words_ratio'] = df['text_tokens'].apply(stopwords_ratio)
    
    df['typos_title_count'] = df['title_tokens'].apply(get_typos_t)
    df['typos_text_count'] = df['text_tokens'].apply(get_typos_t)
    
    df['title_typo_ratio'] = df['typos_title_count']/len(df['title_tokens'])
    df['text_typo_ratio'] = df['typos_text_count']/len(df['text_tokens'])
    
    df['title_length_char'] = data.title.str.len()
    
    df['title_Upper'] = df['title'].str.count(r'[A-Z]')
    df['title_Upper_Ratio'] = df['title_Upper']/df['title_length_char']
    
    df['text_tokens_with_stopwords'] = df['text'].apply(tokenize_text)  
    df['text_stop_words_ratio'] = df['text_tokens_with_stopwords'].apply(stopwords_ratio)
    
    return df[['title_clean','text_clean_without_reuters', 'text_typo_ratio', 'text_stop_words_ratio','title_length_char','typos_text_count', 'typos_title_count', 'title_tokens','text_tokens', 'title_Upper_Ratio', 'score']]

In [22]:
data = clean_data(data)

# Time to run a cross_val and find the best params for the model

## separating the data

In [23]:
x = data[['title_clean', 'text_clean_without_reuters','text_typo_ratio']]
y=data['score']

## Splitting the data in two : Train/Test

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=0.3)

## Creating a pipeline to vectorize the text/title

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
preprocessor = ColumnTransformer([
    
    ('vectorizer_title', CountVectorizer(), 'title_clean'),
    ('vectorizer_text', CountVectorizer(), 'text_clean_without_reuters'),

    #insert function here
    
])

final_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('Logistic', LogisticRegression())])

## Grid searching for best params 

In [26]:
from sklearn.model_selection import GridSearchCV

parameters = {
    
    'Logistic__solver': ('newton-cg', 'lbfgs', 'sag'),
    'Logistic__C': ([0.2, 0.4, 0.6, 0.8, 1.0])

}

grid_search = GridSearchCV(final_pipe,
                           parameters,
                           scoring = ["f1", "accuracy", "recall"], 
                           refit= "accuracy",
                           cv=3,
                           verbose = 0)

In [43]:
import time
start = time.time()
grid_search.fit(x_train,y_train)
stop = time.time()

c:\users\liamc\.venvs\lewagon\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\liamc\.venvs\lewagon\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\liamc\.venvs\lewagon\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\liamc\.venvs\lewagon\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\liamc\.venvs\lewagon\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which 

In [45]:
grid_search.best_params_

{'Logistic__C': 0.4, 'Logistic__solver': 'newton-cg'}

**ok so now that we have best params lets train a model on them**

### Training model on best params found in grid_search with reuters in text

In [26]:
final_pipe.fit(x_train,y_train)

c:\users\liamc\.venvs\lewagon\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('preprocessing', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('vectorizer_title', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', ...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [27]:
final_pipe.score(x_test,y_test)

0.9939123979213066

In [29]:
data

,title_clean,text_clean_without_reuters,text_typo_ratio,text_stop_words_ratio,title_length_char,typos_text_count,typos_title_count,title_tokens,text_tokens,title_Upper_Ratio,score
0,as us budget fight looms republicans flip thei...,washington the head of a conservative republic...,0.000334,0.357810,64,15,0,"[as, us, budget, fight, looms, republicans, fl...","[washington, the, head, of, a, conservative, r...",0.062500,1
1,us military to accept transgender recruits on ...,washington transgender people will be allowed ...,0.000223,0.349359,64,10,0,"[us, military, to, accept, transgender, recrui...","[washington, transgender, people, will, be, al...",0.062500,1
2,senior us republican senator 'let mr mueller d...,washington the special counsel investigation o...,0.000178,0.382932,60,8,1,"[senior, us, republican, senator, 'let, mr, mu...","[washington, the, special, counsel, investigat...",0.116667,1
3,fbi russia probe helped by australian diplomat...,washington trump campaign adviser george papad...,0.000379,0.361702,59,17,3,"[fbi, russia, probe, helped, by, australian, d...","[washington, trump, campaign, adviser, george,...",0.135593,1
4,trump wants postal service to charge 'much mor...,seattlewashington president donald trump calle...,0.000512,0.372066,69,23,1,"[trump, wants, postal, service, to, charge, 'm...","[seattlewashington, president, donald, trump, ...",0.057971,1
5,white house congress prepare for talks on spen...,west palm beach flawashington the white house ...,0.000356,0.347245,64,16,0,"[white, house, congress, prepare, for, talks, ...","[west, palm, beach, flawashington, the, white,...",0.046875,1
6,trump says russia probe will be fair but timel...,west palm beach fla president donald trump sai...,0.000200,0.371747,63,9,1,"[trump, says, russia, probe, will, be, fair, b...","[west, palm, beach, fla, president, donald, tr...",0.079365,1
7,factbox trump on twitter dec approval rating a...,the following statements were posted to the ve...,0.000156,0.318519,60,7,1,"[factbox, trump, on, twitter, dec, approval, r...","[the, following, statements, were, posted, to,...",0.100000,1
8,trump on twitter dec global warming,the following statements were posted to the ve...,0.000134,0.310000,42,6,0,"[trump, on, twitter, dec, global, warming]","[the, following, statements, were, posted, to,...",0.119048,1
9,alabama official to certify senatorelect jones...,washington alabama secretary of state john mer...,0.000067,0.358209,76,3,2,"[alabama, official, to, certify, senatorelect,...","[washington, alabama, secretary, of, state, jo...",0.078947,1


In [30]:
# this final pipe is fitted with training data cointaing reuters in the text

## Testing our model on some new data

## Creating a function that cleans the test text

In [31]:
import trafilatura
downloaded = trafilatura.fetch_url('https://www.rte.ie/news/ireland/2020/1121/1179617-zoo-funding/')

In [32]:
text_test = trafilatura.extract(downloaded)
title_test='€1.6m in funding secured to support zoo sector'

In [33]:
import string

punc = string.punctuation + '“' + '”' + '’' + '‘' +'—' +'€' + '❤'

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def clean_text(text):
    for punctuation in punc:
        text = text.replace(punctuation, '')
    text = ''.join(word for word in text if not word.isdigit())
    text = text.lower()
    text_split = text.split()
    for word in text_split:
        if word in stop_words:
            text_split.remove(word)
    text_string = ''
    for word in text_split:
        text_string = text_string + ' ' + word
    return text_string

In [34]:
text_clean = clean_text(text_test)
title_clean = clean_text(title_test)

## Creating a general function to calculate typo ratio

In [35]:
def typo_ratio(text):
    text_split = text.split()
    lenght = len(text_split)
    typos = get_typos_t(text_split)
    return typos/lenght

## Transforming the data as an input that the model accepts

In [36]:
test_series=pd.Series(text_clean)
title_series=pd.Series(title_clean)

In [37]:
test_df=pd.DataFrame({'title_clean':title_series,'text_clean':test_series, 'text_typo_ratio': typo_ratio(text_clean)})

In [38]:
final_pipe.predict(test_df)

ValueError: Column ordering must be equal for fit and for transform when using the remainder keyword

In [35]:
# this is how we use the model to predict on one specific example(this pipe is fitted with reuters text)

**okay so this model is generalising alot better**

# Checking how well the model generalise for fake news using the fake news only dataset 

In [ ]:
fake_test = pd.read_csv('../raw_data/fake_extra.csv')
fake_test['score'] = 0
fake_test = clean_data(fake_test)

In [ ]:
result_fake = final_pipe.predict(fake_test)
unique_fake, counts_fake = np.unique(result_fake, return_counts=True)
dict(zip(unique_fake, counts_fake))

In [41]:
4140/(4140+562)

0.880476393024245

In [70]:
#this is the accuracy on fake dataset by the pipeline fitted without reuters

## Testing pipeline on true news 

In [ ]:
true_test = pd.read_csv('../raw_data/new_york_real.csv')
true_test['score'] = 1
true_test = clean_data(true_test)

In [40]:
result_true = final_pipe.predict(true_test)
unique_true, counts_true = np.unique(result_true, return_counts=True)
dict(zip(unique_true, counts_true))

{0: 4931, 1: 2872}

In [42]:
2872/(4931+2872)

0.3680635652953992

# Trying a logistic regression with more features

## Iterating over different features that we saw were relevant to distinguish fake from real news with a better score

In [54]:
data.columns

Index(['title_clean', 'text_clean_without_reuters', 'text_typo_ratio',
       'text_stop_words_ratio', 'title_length_char', 'typos_text_count',
       'typos_title_count', 'title_tokens', 'text_tokens', 'title_Upper_Ratio',
       'score'],
      dtype='object')

In [39]:
x_iter= data[['title_clean', 'text_clean_without_reuters','title_length_char','title_Upper_Ratio', 'text_typo_ratio','text_stop_words_ratio']]
y_iter=data['score']

In [40]:
from sklearn.model_selection import train_test_split
x_train_iter,x_test_iter,y_train_iter,y_test_iter =train_test_split(x_iter,y_iter,random_state=0,test_size=0.3)

In [41]:
import re
def rem_urls(text):
    return re.sub('https?:\S+','',text)

import string
punc_no_sq = '!“#$%&\()*+,./:;<=>?@[\\]^_`{|}~“”—’-'
def remove_punctuation(text):
    for punctuation in punc_no_sq:
        text = text.replace(punctuation, '')
    return text

def remove_numbers(text):
    text = ''.join(word for word in text if not word.isdigit())
    return text

def lower_case(text):
    text = text.lower()
    return text

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


def tokenize_text(text):
    return text.split()

from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def stopwords_ratio(tokens):
    count_stop_words = 0
    amount_tokens = 0   
    for token in tokens:
        amount_tokens += 1
        if token in stop_words:
            count_stop_words += 1
    if amount_tokens == 0:
        return 0
    return count_stop_words / amount_tokens

import enchant
english = enchant.DictWithPWL("en_US", "vocab.txt")
wrong_words={}
correct_words=set()
def get_typos_t(tokens):
     wrong_count=0
     for token in tokens:
            if token in wrong_words:
                wrong_words[token]+=1
                wrong_count+=1
            else:
                if not token in correct_words:
                    if token[0].islower() and not '-' in token and not english.check(token) and not english.check(token.capitalize()):
                        wrong_words[token]=1
                        wrong_count+=1
                    else:
                        correct_words.add(token)
     return wrong_count 


def clean_data(data):
    
    df = data.copy()
    
    #drop nan values in df
    
    df.dropna(axis = 0, inplace = True)
    
    #add title_clean and text_clean to df
    stop_words = ['reuters']    
    pat = '|'.join(r"\b{}\b".format(x) for x in stop_words)
    
    df['title_clean'] = df['title'].apply(rem_urls)
    df['text_clean_without_reuters'] = df['text'].apply(rem_urls)
    df['text_clean_without_reuters'] = df['text_clean_without_reuters'].str.replace(pat, '')
    df['title_clean'] = df['title_clean'].apply(remove_punctuation)
    df['text_clean_without_reuters'] = df['text_clean_without_reuters'].apply(remove_punctuation)
    df['title_clean'] = df['title_clean'].apply(remove_numbers)
    df['text_clean_without_reuters'] = df['text_clean_without_reuters'].apply(remove_numbers)
    df['title_clean'] = df['title_clean'].apply(lower_case)
    df['text_clean_without_reuters'] = df['text_clean_without_reuters'].apply(lower_case)
    df['title_clean'] = df['title_clean'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop_words)]))
    df['text_clean_without_reuters'] = df['text_clean_without_reuters'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop_words)]))
    
    #adding features to the df
    
    df['title_tokens'] = df['title_clean'].apply(tokenize_text)
    df['text_tokens'] = df['text_clean_without_reuters'].apply(tokenize_text)
    
    df['text_stop_words_ratio'] = df['text_tokens'].apply(stopwords_ratio)
    
    df['typos_title_count'] = df['title_tokens'].apply(get_typos_t)
    df['typos_text_count'] = df['text_tokens'].apply(get_typos_t)
    
    df['title_typo_ratio'] = df['typos_title_count']/len(df['title_tokens'])
    df['text_typo_ratio'] = df['typos_text_count']/len(df['text_tokens'])
    
    df['title_length_char'] = data.title.str.len()
    
    df['title_Upper'] = df['title'].str.count(r'[A-Z]')
    df['title_Upper_Ratio'] = df['title_Upper']/df['title_length_char']
    
    df['text_tokens_with_stopwords'] = df['text'].apply(tokenize_text)  
    df['text_stop_words_ratio'] = df['text_tokens_with_stopwords'].apply(stopwords_ratio)
    
    return df[['title_clean', 'text_clean_without_reuters','title_length_char','title_Upper_Ratio', 'text_typo_ratio','text_stop_words_ratio','score']]

## Let's train a model on the different features we have

In [42]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

preprocessor_iter = ColumnTransformer([
    
    ('vectorizer_title', CountVectorizer(), 'title_clean'),
    ('vectorizer_text', CountVectorizer(), 'text_clean_without_reuters'),
    ('scaling_title_char', MinMaxScaler(), ['title_length_char']),
   
    #insert function here
])

final_pipe_iter = Pipeline([
    ('preprocessing', preprocessor_iter),
    ('Logistic', LogisticRegression(solver = 'newton-cg', C =0.4 ))])

In [43]:
final_pipe_iter.fit(x_train_iter,y_train_iter)

c:\users\liamc\.venvs\lewagon\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Pipeline(memory=None,
     steps=[('preprocessing', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('vectorizer_title', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', ...ty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False))])

In [44]:
first_test_new_york = pd.read_csv('../raw_data/new_york_real.csv')
first_test_new_york['score'] = 1

In [45]:
result_iter = final_pipe_iter.predict(clean_data(first_test_new_york))
unique_iter, counts_iter = np.unique(result_iter, return_counts=True)
dict(zip(unique_iter, counts_iter))

{0: 4896, 1: 2907}

In [117]:
#combinations tried so far :
# all features - 2907 real news detected
# no features - 2895 real news detected
# title_Upper ratio alone - 
# title_len_char - 
#text_typo + title_Upper_Ratio - 2895
#text_typo 2872 real news detected

#title_len_char + title_Upper_Ratio, text_typo_ratio, text_stop_words_ratio = best features 

# Adding more data for our model to generalize better

In [46]:
margarida_fake = pd.read_csv('../raw_data/fake_extra.csv')
margarida_fake.dropna(inplace = True)
margarida_fake['score'] = 0

In [47]:
margarida_fake = clean_data(margarida_fake)

In [48]:
business_insider_true = pd.read_csv('../raw_data/Business_Insider.csv', nrows = 5000)
business_insider_true.dropna(inplace = True)
business_insider_true['score'] = 1

In [50]:
business_insider_true = clean_data(business_insider_true)

In [51]:
washington_post_true = pd.read_csv('../raw_data/Washington_Post.csv', nrows = 5000)
washington_post_true.dropna(inplace = True)
washington_post_true['score'] = 1

In [52]:
washington_post_true = clean_data(washington_post_true)

In [53]:
data_new = pd.concat([data,margarida_fake,business_insider_true,washington_post_true],ignore_index=True, sort = True)

In [54]:
x_new= data_new[['title_clean', 'text_clean_without_reuters','title_length_char','title_Upper_Ratio', 'text_typo_ratio','text_stop_words_ratio']]
y_new=data_new['score']

In [58]:
from sklearn.model_selection import train_test_split
x_train_new,x_test_new,y_train_new,y_test_new =train_test_split(x_new,y_new,random_state=0,test_size=0.3)

In [59]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

preprocessor_new = ColumnTransformer([
    
    ('vectorizer_title', CountVectorizer(), 'title_clean'),
    ('vectorizer_text', CountVectorizer(), 'text_clean_without_reuters'),
    ('scaling_title_char', MinMaxScaler(), ['title_length_char']),
   
    #insert function here
])

final_pipe_new = Pipeline([
    ('preprocessing', preprocessor_iter),
    ('Logistic', LogisticRegression(solver = 'newton-cg', C =0.4 ))])

In [60]:
final_pipe_new.fit(x_train_new,y_train_new)

c:\users\liamc\.venvs\lewagon\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Pipeline(memory=None,
     steps=[('preprocessing', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('vectorizer_title', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', ...ty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False))])

In [64]:
guardian_true = pd.read_csv('../raw_data/Guardian.csv')
guardian_true['score'] = 1

In [65]:
guardian_true = clean_data(guardian_true)
guardian_true.drop(columns =['score'], inplace = True)

In [67]:
predict = final_pipe_new.predict(guardian_true)

In [68]:
count_1 = 0
count = 0
for pred in predict:
    count += 1
    if pred == 1:
        count_1 += 1
print(count_1/count)

0.8471638655462185


In [69]:
new_fake = pd.read_csv('../raw_data/fake_new_test.csv')
new_fake['score'] = 0
new_fake = clean_data(new_fake)
new_fake.drop(columns =['score'], inplace = True)
predict = final_pipe_new.predict(new_fake)

In [72]:
count_0 = 0
count = 0
for pred in predict:
    count += 1
    if pred == 0:
        count_0 += 1
print(count_0/count)

0.8445006321112516


In [73]:
new_true_1 = pd.read_csv('../raw_data/CNN_test.csv')
new_true_1['score'] = 1
new_true_1 = clean_data(new_true_1)
new_true_1.drop(columns =['score'], inplace = True)
predict = final_pipe_new.predict(new_true_1)

In [74]:
count_1 = 0
count = 0
for pred in predict:
    count += 1
    if pred == 1:
        count_1 += 1
print(count_1/count)

0.8958043175487466


In [75]:
new_true_2 = pd.read_csv('../raw_data/Vox.csv')
new_true_2['score'] = 1
new_true_2 = clean_data(new_true_2)
new_true_2.drop(columns =['score'], inplace = True)
predict = final_pipe_new.predict(new_true_2)

In [76]:
count_1 = 0
count = 0
for pred in predict:
    count += 1
    if pred == 1:
        count_1 += 1
print(count_1/count)

0.8502122498483929


In [77]:
new_fake_1 = pd.read_csv('../raw_data/FAKE_3.csv')
new_fake_1['score'] = 0
new_fake_1 = clean_data(new_fake_1)
new_fake_1.drop(columns =['score'], inplace = True)
predict = final_pipe_new.predict(new_fake_1)

In [78]:
count_0 = 0
count = 0
for pred in predict:
    count += 1
    if pred == 0:
        count_0 += 1
print(count_0/count)

0.16957605985037408


In [5]:
test = pd.read_csv('../raw_data/datasets_concat.csv')

In [8]:
test = test[['title', 'text', 'score']]

In [10]:
test.to_csv(r'../raw_data/all_data.csv')

In [11]:
test = pd.read_csv('../raw_data/all_data.csv')

In [13]:
test.shape

(57183, 4)

In [17]:
test = test[['title', 'text', 'score']]
test.to_csv(r'../raw_data/working.csv', index = False)
test = pd.read_csv('../raw_data/working.csv')
test['score'].value_counts()

1    29000
0    28183
Name: score, dtype: int64